<a href="https://colab.research.google.com/github/RaymondYG/cs246-Mining-Massive-Data-Sets/blob/main/Copy_of_CS246_Colab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 1
## Word Count in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 45.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=df7ed29165c3ae03a252de83de7c24b1d7cf1febcdec57463a777cfb68484e47
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you successfully run the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter, we want to count the total number of (non-unique) words that start with a specific letter.

In your implementation, **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all words that **start** with a non-alphabetic character. You should output word counts for the **entire document**, inclusive of the title, author, and the main texts. If you encounter words broken as a result of new lines, e.g. "pro-ject" where the segment after the dash sign is on a new line, no special processing is needed and you can safely consider it as two words.

Your outputs will be graded on a range -- if your differences from the ground-truths are within an error threshold of 5, you'll be considered correct.

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [6]:
# YOUR
txt = spark.read.text("pg100.txt")
txt.show()

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
| William Shakespeare|
|                    |
|This eBook is for...|
|almost no restric...|
|re-use it under t...|
|with this eBook o...|
|                    |
|** This is a COPY...|
|**     Please fol...|
|                    |
|Title: The Comple...|
|                    |
|Author: William S...|
|                    |
|Posting Date: Sep...|
|Release Date: Jan...|
|                    |
|   Language: English|
|                    |
+--------------------+
only showing top 20 rows



In [22]:
# CODE
from operator import add
import string

rdd = txt.rdd.map(lambda r: r[0])

rdd_split = rdd.flatMap(lambda x:x.split(' '))
rdd_filter = rdd_split.filter(lambda x:x not in string.punctuation)
rdd_map = rdd_filter.map(lambda x: (x.lower(),1))
rdd_reduce = rdd_map.reduceByKey(add)
output = rdd_reduce.collect()

In [25]:
# HERE
# take second element for sort
def takeSecond(elem):
    return elem[1]
output.sort(key=takeSecond,reverse = True)
print(output[:10])

[('the', 27730), ('and', 26099), ('i', 19540), ('to', 18762), ('of', 18126), ('a', 14436), ('my', 12456), ('in', 10730), ('you', 10696), ('that', 10501)]


Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!